In [ ]:
# Course : CS 513 - Knowledge Discovery and Data Mining
# Group Members: Branden Bulatao, Joseph Faustino, Natalie Fortes, Isabel Sutedjo
# Id : 20005971, 20006114, 20006007, 20006618
# Purpose : Music Happiness Predictor - Predicts the happiness of music tracks based on various features.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("dataset.csv")

# Focus on top 5 genres
top_genres = df['track_genre'].value_counts().nlargest(5).index
df = df[df['track_genre'].isin(top_genres)]

# Select relevant features
features = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'valence', 'tempo']
X = df[features]
y = df['track_genre']

# Drop missing values
X = X.dropna()
y = y[X.index]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Base estimator (Decision Tree with max depth to prevent overfitting)
base_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5)

# Boosted model (C5.0-like)
model = AdaBoostClassifier(estimator=base_tree, n_estimators=50, learning_rate=1.0)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("C5.0-style (Boosted) Decision Tree Report:")
print(classification_report(y_test, y_pred))

C5.0-style (Boosted) Decision Tree Report:
              precision    recall  f1-score   support

    acoustic       0.63      0.56      0.60       216
    afrobeat       0.58      0.56      0.57       190
    alt-rock       0.29      0.32      0.30       192
 alternative       0.36      0.36      0.36       210
     ambient       0.78      0.82      0.80       192

    accuracy                           0.52      1000
   macro avg       0.53      0.52      0.53      1000
weighted avg       0.53      0.52      0.52      1000

